In [21]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog
import matplotlib.pyplot as plt

In [32]:
path = '/Users/davidmiles-skov/Desktop/Academics/Projects/market-clearing-price/data/test.csv'
df = pd.read_csv(path)
df

,Company,Supply/Demand,ID,Quantity (MWh),Price (€/MWh)
0,RT R© Supply,Supply,G1,120,0.0
1,WeTrustInWind,Supply,G2,50,0.0
2,BlueHydro,Supply,G3,200,15.0
3,RT R© Supply,Supply,G4,400,30.0
4,KøbenhavnCHP,Supply,G5,60,32.5
5,KøbenhavnCHP,Supply,G6,50,34.0
6,KøbenhavnCHP,Supply,G7,60,36.0
7,DirtyPower,Supply,G8,100,37.5
8,DirtyPower,Supply,G9,70,39.0
9,DirtyPower,Supply,G10,50,40.0


Gathering data

In [33]:
supply_price = df[df['Supply/Demand']=='Supply']['Price (€/MWh)'].to_numpy()
supply_quantity = df[df['Supply/Demand']=='Supply']['Quantity (MWh)'].to_numpy()
demand_price = df[df['Supply/Demand']=='Demand']['Price (€/MWh)'].to_numpy()
demand_quantity = df[df['Supply/Demand']=='Demand']['Quantity (MWh)'].to_numpy()

N_G, N_D = len(supply_price), len(demand_price)
N_G, N_D

(15, 12)

Setting up variables for linprog

In [70]:
c = np.append(supply_price,-demand_price)
A_eq = np.array([np.append(np.ones(N_G), -np.ones(N_D))])
b_eq = 0
A = np.identity(N_G+N_D)
b = np.append(supply_quantity,demand_quantity)

linprog

In [78]:
result = linprog(c, A_eq=A_eq, b_eq=b_eq, A_ub=A, b_ub=b)

Checking answer

In [79]:
result

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -94817.5
              x: [ 1.200e+02  5.000e+01 ...  0.000e+00  0.000e+00]
            nit: 1
          lower:  residual: [ 1.200e+02  5.000e+01 ...  0.000e+00
                              0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00 ...  1.350e+01
                              2.150e+01]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: [ 0.000e+00]
                 marginals: [ 3.750e+01]
        ineqlin:  residual: [ 0.000e+00  0.000e+00 ...  2.500e+01
                              1.000e+01]
                 marginals: [-3.750e+01 -3.750e+01 ... -0.000e+00
                             -0.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
  

check lagrange multipliers

In [80]:
result.ineqlin

  residual: [ 0.000e+00  0.000e+00 ...  2.500e+01  1.000e+01]
 marginals: [-3.750e+01 -3.750e+01 ... -0.000e+00 -0.000e+00]

In [81]:
result.eqlin

  residual: [ 0.000e+00]
 marginals: [ 3.750e+01]